In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json
from tqdm import tqdm

tqdm.pandas()

In [3]:
lib_df = pd.read_csv("data/mce_library_first_metadata.csv")

In [4]:
lib_df.head()

,RackCode,Plate Location,VialCode,Cat. No.,Product Name,Synonyms,CAS No.,M.Wt,Target,Saltdata,...,Formula,Smiles,Solubility,Solvent,Batch No.,Quantity,URL,Pathway,Research Area,Clinical Information
0,HYCPK16574,A2,NaN,HY-15338,TG003,NaN,719277-26-6,249.33,CDK,Free Base,...,C13H15NO2S,CC(/C=C1SC2=CC=C(C=C2N\1CC)OC)=O.[Z],DMSO : ≥ 31 mg/mL (124.33 mM),DMSO Solution,21769,10mM * 30uL,https://www.medchemexpress.com/TG003.html,Cell Cycle/DNA Damage,Cancer,No Development Reported
1,HYCPK16574,A3,NaN,HY-15440B,Fostemsavir Tris,BMS-663068 (Tris),864953-39-9,704.62,HIV,Tris,...,C29H37N8O11P,O=C(N1CCN(C(C2=CC=CC=C2)=O)CC1)C(C3=CN(COP(O)(...,DMSO : 125 mg/mL (ultrasonic);H2O : 100 mg/mL ...,DMSO Solution,58208,10mM * 30uL,https://www.medchemexpress.com/BMS-663068-Tris...,Anti-infection,Infection,Launched
2,HYCPK16574,A4,NaN,HY-114315,NQO1 substrate,NaN,2304503-05-5,268.18,Others,Free Base,...,C13H2F2N4O,N#CC1=C(C#N)N=C2C(C(C3=C2C=C(F)C(F)=C3)=O)=N1,DMSO : 125 mg/mL (466.10 mM; Need ultrasonic),DMSO Solution,63377,10mM * 30uL,https://www.medchemexpress.com/nqo1-substrate....,Others,Cancer,No Development Reported
3,HYCPK16574,A5,NaN,HY-15357,ALK inhibitor 1,NaN,761436-81-1,562.48,ALK; FAK,Free Base,...,C23H28BrN7O3S,O=S(C1=C(NC2=C(Br)C=NC(NC3=C(OC)C=C(N4CCN(C)CC...,H2O : < 0.1 mg/mL (insoluble); DMSO : ≥ 100 mg...,DMSO Solution,6410,10mM * 30uL,https://www.medchemexpress.com/ALK-inhibitor-1...,Protein Tyrosine Kinase/RTK,Cancer,No Development Reported
4,HYCPK16574,A6,NaN,HY-15880,CCT007093,NaN,176957-55-4,272.39,Apoptosis; Autophagy; Phosphatase,Free Base,...,C15H12OS2,O=C1/C(CC/C1=C\C2=CC=CS2)=C/C3=CC=CS3,DMSO : ≥ 2.8 mg/mL (10.28 mM); DMF : 3.33 mg/m...,DMSO Solution,11223,10mM * 30uL,https://www.medchemexpress.com/CCT007093.html,Apoptosis; Autophagy; Metabolic Enzyme/Protease,Cancer,No Development Reported


In [17]:
# lib_df[["WellLetter", "WellNumber"]] = lib_df["Plate Location"].str.split(expand=True)
# lib_df["WellLetter"] = [value[:1] for value in lib_df["Plate Location"]]
# lib_df["WellNumber"] = [int(value[1:]) for value in lib_df["Plate Location"]]
# lib_df.drop(columns=["WellNumber", "WellLetter"], inplace=True)
lib_df[["WellNumber", "WellLetter"]] = [[int(value[1:]),value[:1]] for value in lib_df["Plate Location"]]

In [18]:
sorted_df = lib_df.sort_values(by=["RackCode", "WellLetter", "WellNumber"], ascending=[True, True, True])
sorted_df

,RackCode,Plate Location,VialCode,Cat. No.,Product Name,Synonyms,CAS No.,M.Wt,Target,Saltdata,...,Solubility,Solvent,Batch No.,Quantity,URL,Pathway,Research Area,Clinical Information,WellNumber,WellLetter
0,HYCPK16574,A2,NaN,HY-15338,TG003,NaN,719277-26-6,249.33,CDK,Free Base,...,DMSO : ≥ 31 mg/mL (124.33 mM),DMSO Solution,21769,10mM * 30uL,https://www.medchemexpress.com/TG003.html,Cell Cycle/DNA Damage,Cancer,No Development Reported,2,A
1,HYCPK16574,A3,NaN,HY-15440B,Fostemsavir Tris,BMS-663068 (Tris),864953-39-9,704.62,HIV,Tris,...,DMSO : 125 mg/mL (ultrasonic);H2O : 100 mg/mL ...,DMSO Solution,58208,10mM * 30uL,https://www.medchemexpress.com/BMS-663068-Tris...,Anti-infection,Infection,Launched,3,A
2,HYCPK16574,A4,NaN,HY-114315,NQO1 substrate,NaN,2304503-05-5,268.18,Others,Free Base,...,DMSO : 125 mg/mL (466.10 mM; Need ultrasonic),DMSO Solution,63377,10mM * 30uL,https://www.medchemexpress.com/nqo1-substrate....,Others,Cancer,No Development Reported,4,A
3,HYCPK16574,A5,NaN,HY-15357,ALK inhibitor 1,NaN,761436-81-1,562.48,ALK; FAK,Free Base,...,H2O : < 0.1 mg/mL (insoluble); DMSO : ≥ 100 mg...,DMSO Solution,6410,10mM * 30uL,https://www.medchemexpress.com/ALK-inhibitor-1...,Protein Tyrosine Kinase/RTK,Cancer,No Development Reported,5,A
4,HYCPK16574,A6,NaN,HY-15880,CCT007093,NaN,176957-55-4,272.39,Apoptosis; Autophagy; Phosphatase,Free Base,...,DMSO : ≥ 2.8 mg/mL (10.28 mM); DMF : 3.33 mg/m...,DMSO Solution,11223,10mM * 30uL,https://www.medchemexpress.com/CCT007093.html,Apoptosis; Autophagy; Metabolic Enzyme/Protease,Cancer,No Development Reported,6,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10264,HYCPK16762,A4,NaN,HY-B0272,Rifampicin,Rifampin; Rifamycin AMP,13292-46-1,822.94,Antibiotic; Bacterial; Influenza Virus,Free Base,...,H2O : 1 mg/mL (1.22 mM; ultrasonic and heat to...,DMSO Solution,96905,10mM * 30uL,https://www.medchemexpress.com/rifampicin.html,Anti-infection,Infection,Launched,4,A
10265,HYCPK16762,A5,NaN,HY-Y0546,Benzophenone,NaN,119-61-9,182.22,Endogenous Metabolite,Free Base,...,10 mM in DMSO,DMSO Solution,83850,10mM * 30uL,https://www.medchemexpress.com/benzophenone.html,Metabolic Enzyme/Protease,Metabolic Disease,No Development Reported,5,A
10266,HYCPK16762,A6,NaN,HY-B1008,4-Aminobenzoic acid,PABA; Vitamin Bx; Vitamin H1,150-13-0,137.14,Endogenous Metabolite,Free Acid,...,H2O : 4.55 mg/mL (33.18 mM; Need ultrasonic); ...,DMSO Solution,57774,10mM * 30uL,https://www.medchemexpress.com/4-Aminobenzoic-...,Metabolic Enzyme/Protease,Others,No Development Reported,6,A
10267,HYCPK16762,A7,NaN,HY-N0115,Gastrodin,Gastrodine,62499-27-8,286.28,Others,Free Base,...,DMSO : ≥ 100 mg/mL (349.31 mM),DMSO Solution,26958,10mM * 30uL,https://www.medchemexpress.com/Gastrodin.html,Others,Inflammation/Immunology; Neurological Disease,Launched,7,A


In [31]:
def next_well(well, letter, plate):
    well += 1
    if well > 24:
        well = 1

        # next letter
        letter = ord(letter) + 1
        if letter>80:
            letter = 65
            plate += 1

        letter = chr(letter)

    return well, letter, plate


plate = 1
well = 1
letter = "A"
original_letter = "A"

mixed_plates = []
mixed_wells = []

for index, row in sorted_df.iterrows():
    tmp_letter = row["WellLetter"]
    if tmp_letter is not original_letter:
        original_letter = tmp_letter
        well, letter, plate = next_well(well, letter, plate)

    mixed_plates.append(plate)
    mixed_wells.append("{}{}".format(letter, well))

sorted_df["MixedPlate"] = mixed_plates
sorted_df["MixedWell"] = mixed_wells
sorted_df["mixed_location_plate1"] = ["1D{}_{}".format(plate, well) for well, plate in zip(mixed_wells, mixed_plates)]
sorted_df["mixed_location_plate2"] = ["2D{}_{}".format(plate, well) for well, plate in zip(mixed_wells, mixed_plates)]
sorted_df["mixed_location_plate3"] = ["3D{}_{}".format(plate, well) for well, plate in zip(mixed_wells, mixed_plates)]
sorted_df

,RackCode,Plate Location,VialCode,Cat. No.,Product Name,Synonyms,CAS No.,M.Wt,Target,Saltdata,...,Pathway,Research Area,Clinical Information,WellNumber,WellLetter,MixedWell,MixedPlate,mixed_location_plate1,mixed_location_plate2,mixed_location_plate3
0,HYCPK16574,A2,NaN,HY-15338,TG003,NaN,719277-26-6,249.33,CDK,Free Base,...,Cell Cycle/DNA Damage,Cancer,No Development Reported,2,A,A1,1,1D1_A1,2D1_A1,3D1_A1
1,HYCPK16574,A3,NaN,HY-15440B,Fostemsavir Tris,BMS-663068 (Tris),864953-39-9,704.62,HIV,Tris,...,Anti-infection,Infection,Launched,3,A,A1,1,1D1_A1,2D1_A1,3D1_A1
2,HYCPK16574,A4,NaN,HY-114315,NQO1 substrate,NaN,2304503-05-5,268.18,Others,Free Base,...,Others,Cancer,No Development Reported,4,A,A1,1,1D1_A1,2D1_A1,3D1_A1
3,HYCPK16574,A5,NaN,HY-15357,ALK inhibitor 1,NaN,761436-81-1,562.48,ALK; FAK,Free Base,...,Protein Tyrosine Kinase/RTK,Cancer,No Development Reported,5,A,A1,1,1D1_A1,2D1_A1,3D1_A1
4,HYCPK16574,A6,NaN,HY-15880,CCT007093,NaN,176957-55-4,272.39,Apoptosis; Autophagy; Phosphatase,Free Base,...,Apoptosis; Autophagy; Metabolic Enzyme/Protease,Cancer,No Development Reported,6,A,A1,1,1D1_A1,2D1_A1,3D1_A1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10264,HYCPK16762,A4,NaN,HY-B0272,Rifampicin,Rifampin; Rifamycin AMP,13292-46-1,822.94,Antibiotic; Bacterial; Influenza Virus,Free Base,...,Anti-infection,Infection,Launched,4,A,K19,3,1D3_K19,2D3_K19,3D3_K19
10265,HYCPK16762,A5,NaN,HY-Y0546,Benzophenone,NaN,119-61-9,182.22,Endogenous Metabolite,Free Base,...,Metabolic Enzyme/Protease,Metabolic Disease,No Development Reported,5,A,K19,3,1D3_K19,2D3_K19,3D3_K19
10266,HYCPK16762,A6,NaN,HY-B1008,4-Aminobenzoic acid,PABA; Vitamin Bx; Vitamin H1,150-13-0,137.14,Endogenous Metabolite,Free Acid,...,Metabolic Enzyme/Protease,Others,No Development Reported,6,A,K19,3,1D3_K19,2D3_K19,3D3_K19
10267,HYCPK16762,A7,NaN,HY-N0115,Gastrodin,Gastrodine,62499-27-8,286.28,Others,Free Base,...,Others,Inflammation/Immunology; Neurological Disease,Launched,7,A,K19,3,1D3_K19,2D3_K19,3D3_K19


In [32]:
sorted_df.to_csv("data/mce_first_metadata_formatted.csv")